# Assignment 4

## Competition in Ready-Mix Concrete Manufacturing

![A Concrete Example](pics/readymixConcrete.jpg)

This assignment asks you to study competition in local markets of the Ready-Mix Concrete Manufacturing industry ([NAICS 327320](https://www.census.gov/econ/isp/sampler.php?naicscode=327320&naicslevel=6)). To this end, you can use the script below to estimate a simple version of [Abbring, Campbell, Tilly and Yang's (2017)](https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf) dynamic model of oligopolistic entry, competition, and exit, using data from the U.S. Census Bureau. See e.g. the work of Chad Syverson and that of Allan Collard-Wexler for more information about and economic analyses of the Ready-Mix Concrete Manufacturing industry.

|             | Computation time | mu     | sigma  | k1   | k2   | k3   | k4   | phi   | omega |
|-------------|------------------|--------|--------|------|------|------|------|-------|-------|
| inner=1е-6  | 550              | 0,0034 | 0,0112 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |
| inner=1е-10 | 565              | 0,0034 | 0,0112 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |
| inner=1е-12 | 859              | 0,0034 | 0,0112 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |
| outer=1e-2  | 480              | 0,002  | 0,0154 | 0,4  | 0,27 | 0,23 | 0,18 | 26,44 | 1,56  |
| outer=1e-6  | 565              | 0,0034 | 0,0112 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |
| outer=1e-10 | 1244             | 0,0034 | 0,0112 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |
| Grid=500    | 215              | 0,0034 | 0,0113 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,6   |
| Grid=1000   | 565              | 0,0034 | 0,0112 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |
| Grid=1500   | 2122             | 0,0034 | 0,0111 | 0,38 | 0,26 | 0,22 | 0,17 | 27,72 | 1,59  |

## Script for Estimating a Very Simple Industry Dynamics Model

This script is an adapted version of the script [example.m](example.m) in the Matlab package at [verysimple.abbring.org](http://verysimple.abbring.org). It calls various functions from that package, which we have included with this notebook. Please see [documentation.m.html](documentation.m.html) for detailed documentation of the original script and functions. Here, we only comment on parts of the script that are specific to this assignment.


### Numerical Settings 

In [ ]:
Settings.tolInner = 1e-10;
Settings.tolOuter = 1e-6;
Settings.maxIter = 1000;

Settings.fdStep = 1e-7;

Settings.integrationLength = 32;
[Settings.integrationNodes, Settings.integrationWeights] = ...
    lgwt(Settings.integrationLength, 0, 1);
    
options = optimset( 'Algorithm', 'interior-point', 'Display', 'iter', ...
    'TolFun', Settings.tolOuter, 'TolX', Settings.tolOuter, ...
    'GradObj', 'off');

### Load and Process Data

The script requires the included text file [readymixConcrete.txt](readymixConcrete.txt), which contains annual observations of the numbers of Ready-Mix Concrete Manufacturing plants in 573 local markets (U.S. Micropolitan Statistical Areas) for the 2000–2009 period. It also contains the population of each local market in each year.

We have extracted these data from the Stata data file [markets.dta](markets.dta), which contains information on a range of six-digit NAICS industries (see [marketsDocumentation.txt](marketsDocumentation.txt)). For this, we have used the Stata `do` file [readymixConcrete.do](readymixConcrete.do), which can easily be adapted to extract data for other industries.

In [ ]:
Data = importdata('readymixConcrete.txt',',', 1);
  
totalObs = size(Data.data,1);
Settings.tCheck = max(Data.data(:,2))+1-min(Data.data(:,2));
Settings.rCheck = totalObs/Settings.tCheck;

Data.N = reshape(Data.data(:,3),Settings.tCheck,Settings.rCheck); 
Settings.nCheck = max(max(Data.N));

Data.contC = Data.data(:,4)/10000;

Note that `Data.contC` measures population in tens of thousands of people. We need to discretize the population data on a finite grid. By changing the next three lines of the script, you can control the grid's coarseness and boundaries.  

In [ ]:
Settings.cCheck = 1000;
Settings.lowBndC = 0.9*min(Data.contC);
Settings.uppBndC = 1.1*max(Data.contC);

Settings.logGrid = ...
linspace(log(Settings.lowBndC), log(Settings.uppBndC), Settings.cCheck);
Settings.d = Settings.logGrid(2) - Settings.logGrid(1);

temp = zeros(totalObs,1);
for i = 1:totalObs
    test = (log(Data.contC(i))-Settings.logGrid).^2;
    temp(i) = find(min(test)-test>=0,1);
end
Data.C = reshape(temp,Settings.tCheck,Settings.rCheck);

### Estimation

We compute maximum likelihood estimates using the three-step nested fixed-point procedure outlined in [Abbring et al. (2017)](https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf). 

#### Step 1
The first step estimates the parameters of the population process directly from the population data.

In [ ]:
from = Data.C(1:Settings.tCheck - 1, 1:Settings.rCheck);
to = Data.C(2:Settings.tCheck, 1:Settings.rCheck);

logTransitions = log(Data.C([from(:), to(:)]));
innovLogC = logTransitions(:, 2) - logTransitions(:, 1);
startValues.step1 = [mean(innovLogC)  std(innovLogC)];

objFunStep1 = @(estimates) likelihoodStep1(Data, Settings, estimates); 

tic;
[Estimates.step1, llh.step1, exitFlag.step1] = fmincon(objFunStep1, ...
    startValues.step1, [], [], [], [], [-inf, 0], [], [], options);
computingTime.step1 = toc;

#### Step 2
The second step estimates the remaining parameters using a nested fixed-point maximum likelihood procedure, fixing the parameters of the population process to their first step estimates.

By changing the next five lines of the script, you can control the model's specification (note that this Matlab code, unlike the R/C++ code that was used to produce the simulation and empirical results in [Abbring et al. (2017)](https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf), does not allow for market heterogeneity or economic barriers to entry). 

In [ ]:
Param.rho = 1/1.05; 
Settings.estimates2k = @(x) [x(1:3) x(4)*ones(1,Settings.nCheck-3)];
Settings.estimates2phi = @(x) x(5);
Settings.estimates2omega = @(x) x(6);
startValues.step2 = [0.4 0.4 0.4 0.4 25 1.6];

Param = markov(Param, Settings, Estimates.step1(1), Estimates.step1(2));

objFunStep2 = @(estimates) likelihoodStep2(Data, Settings, Param, estimates);

lb = zeros(size(startValues.step2));

tic;
[Estimates.step2,llh.step2,exitFlag.step2] = fmincon(objFunStep2, ...
    startValues.step2, [], [], [], [], lb, [], [], options);
computingTime.step2 = toc;

#### Step 3
The last step maximizes the full likelihood, starting from the previous steps' estimates, to produce efficient estimates and correct standard errors.  

In [ ]:
startValuesStep3 = [Estimates.step2, Estimates.step1];

objFunStep3 = @(estimates) likelihoodStep3(Data, Settings, Param, estimates);

lb = zeros(size(startValuesStep3));
lb(length(startValuesStep3) - 1) = -inf;

tic;
[Estimates.step3, llh.step3, exitFlag.step3] = fmincon(objFunStep3,...
    startValuesStep3, [], [], [], [], lb, [], [], options);
computingTime.step3 = toc;

[~,Estimates.se] = likelihoodStep3(Data, Settings, Param, Estimates.step3);

## Questions
### 1. Does it makes sense to treat Micropolitan Statistical Areas as distinct local markets of the Ready-Mix Concrete Manufacturing industry?

Ready-mixed concrete is the fluid form of concrete delivered to construction sites in barrel-backed mixer
trucks and poured into a desired shape.

It can be delivered at any destination where a truck can travel and can be discharged directly to the workplace or handling equipment at the point of use. (https://www.grandviewresearch.com/industry-analysis/ready-mix-concrete-market)

However, the combination of the industry’s high transport costs and its reliance on the construction sector imply that the ready-mixed “market” is not a singular, nationwide unit, but instead a collection of quasi-independent local geographic markets. 

"Data from the 2002 Commodity Flow Survey indicate that the average shipment distance for the detailed product category of which ready-mixed comprises the majority of shipments (“nonrefractory mortars and concretes”) is only 32 miles." (http://home.uchicago.edu/~syverson/marketsrmc.pdf)

Therefore, it makes sense to treat Micropolitan Statistical Areas as distinct local markets.


### 2. Interpret and discuss the estimation results.

The estimated values of coefficients obtained are as follows:

- $\mu_{C}*10^{2}=0.34$, which refers to the mean of demand growth on the market. This result can be interpreted the following way: on average the population, which represents the demand on the market, growth by 0.34% annually.

- $\sigma_C*10^2=1.12$, providing the variance of the demand growth, i.e. growth of population on the market. From this we can conclude that the growth of population is rather unstable due to a high standard deviation comparing to an average annual growth. This is in line with the findings of Allan Collard-Wexler(https://onlinelibrary.wiley.com/doi/epdf/10.3982/ECTA6877), who noted that since the size of the local construction industry fluctuates by an average of 30%per year,the ready-mix concrete industry experiences large changes in demand.

- $\phi=27.72$, the parameter of the sunk costs to entry $\phi e^{W_t}$. This can be interpreted in comparance to the costs of staying on the market. As the entry costs is almost 28 times higher comparing to the staying costs, that means that we can expect that companies would 'enter' the market for a relatively long period due to the fact that high entry costs make it unprofitable to enter market and leave soon.

- $\omega=1.59$, the scale parameter of log-normal distribution of exponent of the costs shock $W_t$, which gives the idea about the level of variance of the costs shock.

- $k_1*10^4=0.38$. In order to find out the interpretation of this parameter, minor transformation should be conducted. Thus, $\frac{1}{k_1*10^3}=\frac{1}{0.038}=26.3$. This means that 26.3 thousand population would ensure zero surplus for a monopolist on the market.

- $k_2*10^4=0.26$. This coefficient is respective to the previous one but for 2 companies on the market. However, the transformation and consequently, interpretation, whould be slightly different. In particular,  $\frac{k_2}{k_1}=\frac{0.26}{0.38}=0.684$. From here we could conclude that if on the monopolistic market one more company appears, it reduces the expected surplus by almost 32%

- $k_3*10^4=0.22$. Similiarly, the following parameter can be interpreted: $\frac{k_3}{k_2}=\frac{0.22}{0.26}=0.846$. Thus, if one more company enters the market with two companies, it reduces the expected surplus by almost around 15.4%.

- $k_4*10^4=0.17$. Finally: $\frac{k_4}{k_3}=\frac{0.17}{0.22}=0.773$. Thus, if one more company enters the market with three companies, and the number of companies reaches the maximum possible value, it reduces the expected surplus by almost around 23%

It should also be noted that the obtained results stay in line with what we would expect for $k_i$ parameters according to the existing literature. In particular, it can be noted that the speed of surplus reduction is decreasing with each additional compamy entering the market. It can be explained by the establishment of a highly competitive market, resulting from the high substituability of the product between producers.


## Bonus assignments
### 3. Experiment with the numerical settings and discuss.

### 4. Try different model specifications and discuss.

### 5. Estimate the model for different industries and discuss. 

## References

- Abbring, Jaap, Jeffrey Campbell, Jan Tilly and Nan Yang (2017; Revised April 2018). Very simple Markov-perfect industry dynamics: Empirics. Discussion Paper 2017-021. CentER, Tilburg. https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf